In [ ]:
pip install spark-nlp scikit-learn openai pyspark

In [ ]:
import json
import numpy as np
from collections import Counter
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity

import sparknlp
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, BertEmbeddings

# ⚙️ Khởi tạo Spark NLP
spark = sparknlp.start()

# 📥 Đọc dữ liệu từ file JSONL
file_path = "/opt/workspace/gen_1604_formated.jsonl"
with open(file_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# 📌 Trích câu hỏi từ các tin nhắn của user
questions = [
    m["content"].strip()
    for item in data
    for m in item.get("messages", [])
    if m["role"] == "user" and m.get("content")
]

print(f"📦 Tổng số câu hỏi: {len(questions)}")

# Tạo DataFrame Spark từ danh sách câu hỏi
df = spark.createDataFrame([(q,) for q in questions], ["question"])

# ✳️ Tạo Spark NLP pipeline
document_assembler = DocumentAssembler() \
    .setInputCol("question") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

bert = BertEmbeddings.pretrained("bert_base_multilingual_cased", "xx") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setCaseSensitive(True)

pipeline = Pipeline(stages=[document_assembler, tokenizer, bert])
model = pipeline.fit(df)
result = model.transform(df)

# 🔢 Trích embedding trung bình mỗi câu
def extract_avg_embedding(row):
    vecs = [emb.embeddings for emb in row['embeddings']]
    return np.mean(vecs, axis=0) if vecs else np.zeros(768)

embeddings = result.select("embeddings").rdd.map(extract_avg_embedding).collect()

# Chuyển sang numpy array
embedding_matrix = np.array(embeddings)
embedding_matrix = np.nan_to_num(embedding_matrix, nan=0.0, posinf=0.0, neginf=0.0)

# 🔍 Tính cosine similarity và chuẩn hóa khoảng cách
similarity_matrix = cosine_similarity(embedding_matrix)
distance_matrix = 1 - np.clip(similarity_matrix, 0, 1)

# 🧠 Gom nhóm bằng DBSCAN
clustering = DBSCAN(eps=0.5, min_samples=2, metric="precomputed")
labels = clustering.fit_predict(distance_matrix)

# 📊 Gom nhóm các câu hỏi trùng
groups = {}
for label, question in zip(labels, questions):
    if label == -1:
        continue  # Bỏ noise
    groups.setdefault(label, []).append(question)

# 📋 In kết quả nhóm
for i, (k, g) in enumerate(groups.items()):
    print(f"\n🧩 Nhóm {i+1} ({len(g)} câu):")
    for q in g:
        print(" -", q)

# 📈 Thống kê tần suất
print("\n📊 Tần suất các câu hỏi:")
for question, freq in Counter(questions).most_common():
    print(f"{question} | {freq} lần")
